In [1]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/gdrive')

# You can access your Google Drive files in '/content/gdrive/My Drive/'


Mounted at /content/gdrive


In [ ]:
from PIL import Image
import os

# Set the input and output folders
input_folder = "/content/gdrive/MyDrive/openAI/data/유증상_검증/유증상_라벨"
output_folder = "유증상_검증"

# Desired resolution
new_resolution = (480, 360)

# Iterate through subfolders and resize images
for root, dirs, files in os.walk(input_folder):
    for file in files:
        # Check if the file is an image (you can add more image extensions if needed)
        if file.lower().endswith(('.png', '.jpg', '.jpeg')):
            # Create input and output paths
            input_path = os.path.join(root, file)
            output_path = os.path.join(output_folder, os.path.relpath(input_path, input_folder))

            # Create output folder if it doesn't exist
            os.makedirs(os.path.dirname(output_path), exist_ok=True)

            # Open image, resize, and save to the output folder
            with Image.open(input_path) as img:
                img = img.resize(new_resolution, Image.ANTIALIAS)
                img.save(output_path)

print("Resizing complete.")


<ipython-input-4-ec7a059d6ed7>:25: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = img.resize(new_resolution, Image.ANTIALIAS)


Resizing complete.


In [ ]:
!zip -r /content/test.zip /content/유증상_검증

  adding: content/유증상_검증/ (stored 0%)
  adding: content/유증상_검증/A4_농포_여드름/ (stored 0%)
  adding: content/유증상_검증/A4_농포_여드름/IMG_D_A4_182311.jpg (deflated 1%)
  adding: content/유증상_검증/A4_농포_여드름/IMG_D_A4_182265.jpg (deflated 1%)
  adding: content/유증상_검증/A4_농포_여드름/IMG_D_A4_182782.jpg (deflated 1%)
  adding: content/유증상_검증/A4_농포_여드름/IMG_D_A4_182006.jpg (deflated 2%)
  adding: content/유증상_검증/A4_농포_여드름/IMG_D_A4_182953.jpg (deflated 1%)
  adding: content/유증상_검증/A4_농포_여드름/IMG_D_A4_182512.jpg (deflated 1%)
  adding: content/유증상_검증/A4_농포_여드름/IMG_D_A4_182634.jpg (deflated 2%)
  adding: content/유증상_검증/A4_농포_여드름/IMG_D_A4_182748.jpg (deflated 1%)
  adding: content/유증상_검증/A4_농포_여드름/IMG_D_A4_182449.jpg (deflated 1%)
  adding: content/유증상_검증/A4_농포_여드름/IMG_D_A4_182767.jpg (deflated 1%)
  adding: content/유증상_검증/A4_농포_여드름/IMG_D_A4_182065.jpg (deflated 1%)
  adding: content/유증상_검증/A4_농포_여드름/IMG_D_A4_013092.jpg (deflated

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
import os

# Set the input and output folders
input_folder = "/content/drive/MyDrive/openAI/data/유증상_학습"
validation_folder = "/content/drive/MyDrive/openAI/data/유증상_검증"

# Create an ImageDataGenerator with the desired preprocessing
data_generator = ImageDataGenerator(rescale=1./255)
validation_data_generator = ImageDataGenerator(rescale=1./255)

new_resolution=(480,360)
# Flow images from the input folder using the generator, with labels being the subfolders
image_generator = data_generator.flow_from_directory(
    input_folder,
    target_size=new_resolution,
    class_mode='categorical',  # Specify categorical to use subfolders as labels
    batch_size=64,
    shuffle=True,
    seed=42  # Set a seed for reproducibility
)

validation_generator = validation_data_generator.flow_from_directory(
    validation_folder,
    target_size=new_resolution,
    class_mode='categorical',  # Specify categorical to use subfolders as labels
    batch_size=64,
    shuffle=False  # Set shuffle to False for validation
)

# Create a simple CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(new_resolution[0], new_resolution[1], 3), kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.3))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.3))
model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(len(image_generator.class_indices), activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define a callback to save the best weights during training
checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_accuracy', mode='max', verbose=1)

# Train the model using the image generator
model.fit(
    image_generator,
    #steps_per_epoch=image_generator.samples // image_generator.batch_size,
    validation_data=validation_generator,
    epochs=10,  # Adjust the number of epochs based on your needs
    callbacks=[checkpoint]
)


Found 7504 images belonging to 5 classes.
Found 2700 images belonging to 5 classes.
Epoch 1/10
 18/118 [===>..........................] - ETA: 12:47 - loss: 36.2174 - accuracy: 0.2057

In [3]:
!pip show tensorflow

Name: tensorflow
Version: 2.15.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, ml-dtypes, numpy, opt-einsum, packaging, protobuf, setuptools, six, tensorboard, tensorflow-estimator, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: dopamine-rl


In [2]:
!pip install tqdm

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision.models import vgg16
from tqdm import tqdm  # Import tqdm for progress bar

# Set the input and output folders
train_folder = "/content/drive/MyDrive/openAI/data/유증상_학습"
validation_folder = "/content/drive/MyDrive/openAI/data/유증상_검증"

# Desired resolution
new_resolution = (224, 224)  # Adjust based on the input size expected by VGG16

# Define data transformations
transform = transforms.Compose([
    transforms.Resize(new_resolution),
    transforms.ToTensor(),
])

# Create custom dataset classes for training and validation
train_dataset = ImageFolder(train_folder, transform=transform)
validation_dataset = ImageFolder(validation_folder, transform=transform)

# Create DataLoader for training and validation datasets
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
validation_loader = DataLoader(validation_dataset, batch_size=32, shuffle=False, num_workers=4)

# Define the CNN model using VGG16 as a base
class CustomVGG16(nn.Module):
    def __init__(self, num_classes):
        super(CustomVGG16, self).__init__()
        vgg_model = vgg16(pretrained=True)
        self.features = vgg_model.features
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

# Create an instance of the model
num_classes = len(train_dataset.classes)
model = CustomVGG16(num_classes)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Training the model with tqdm
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0
    with tqdm(total=len(train_loader), desc=f"Epoch {epoch + 1}/{num_epochs}", unit="batch") as pbar:
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            pbar.set_postfix({'loss': train_loss / total, 'accuracy': correct / total})
            pbar.update()

    # Validation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in validation_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print(f"Epoch {epoch + 1}/{num_epochs}, Validation Accuracy: {accuracy * 100:.2f}%")

# Save the trained model
torch.save(model.state_dict(), 'best_model.pth')


Epoch 1/10: 100%|██████████| 142/142 [01:09<00:00,  2.04batch/s, loss=1.19, accuracy=0.332]


Epoch 1/10, Validation Accuracy: 38.33%


Epoch 2/10: 100%|██████████| 142/142 [01:07<00:00,  2.11batch/s, loss=1.1, accuracy=0.319]


Epoch 2/10, Validation Accuracy: 39.41%


Epoch 3/10: 100%|██████████| 142/142 [01:06<00:00,  2.13batch/s, loss=1.1, accuracy=0.334]


...

Epoch 8/10, Validation Accuracy: 39.41%


Epoch 9/10: 100%|██████████| 142/142 [01:06<00:00,  2.12batch/s, loss=1.1, accuracy=0.327]


Epoch 9/10, Validation Accuracy: 39.41%


Epoch 10/10: 100%|██████████| 142/142 [01:07<00:00,  2.10batch/s, loss=1.1, accuracy=0.329]


Epoch 10/10, Validation Accuracy: 39.41%
